<a href="https://colab.research.google.com/github/wangshouh/LearnDataCleaning/blob/main/cleardata_ch1_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pymssql
!pip install mysql-connector-python

     |████████████████████████████████| 37.5 MB 53 kB/s 


In [5]:
import pandas as pd
import numpy as np
import pymssql
import mysql.connector

In [14]:
query = "SELECT studentid, school, sex, age, famsize, medu AS mothereducation, fedu AS fathereducation, traveltime, studytime, failures, famrel, freetime,\
goout, g1 AS gradeperiod1, g2 AS gradeperiod2,g3 AS gradeperiod3 From studentmath"

server = "pdcc.c9sqqzd5fulv.us-west-2.rds.amazonaws.com"
user = "pdccuser"
password = "pdccpass"
database = "pdcctest"

conn = pymssql.connect(server=server, user=user, password=password, database=database)
studentmath = pd.read_sql(query, conn)
conn.close()

In [15]:
studentmath.head()

,studentid,school,sex,age,famsize,mothereducation,fathereducation,traveltime,studytime,failures,famrel,freetime,goout,gradeperiod1,gradeperiod2,gradeperiod3
0,001,GP,F,18,GT3,4,4,2,2,0,4,3,4,5,6,6
1,002,GP,F,17,GT3,1,1,1,2,0,5,3,3,5,5,6
2,003,GP,F,15,LE3,1,1,1,2,3,4,3,2,7,8,10
3,004,GP,F,15,GT3,4,2,1,3,0,3,2,2,15,14,15
4,005,GP,F,16,GT3,3,3,1,2,0,4,3,2,6,10,10


In [16]:
studentmath.dtypes

studentid          object
school             object
sex                object
age                 int64
famsize            object
mothereducation     int64
fathereducation     int64
traveltime          int64
studytime           int64
failures            int64
famrel              int64
freetime            int64
goout               int64
gradeperiod1        int64
gradeperiod2        int64
gradeperiod3        int64
dtype: object

In [17]:
newcolorder = ['studentid', 'gradeperiod1','gradeperiod2','gradeperiod3', 'school', 'sex', 'age', 'famsize','mothereducation', 'fathereducation', 'traveltime','studytime', 'freetime', 'failures', 'famrel','goout']
studentmath = studentmath[newcolorder]

In [24]:
studentmath.studentid.count() == studentmath.studentid.nunique()

True

In [27]:
studentmath.set_index('studentid', inplace=True)

In [25]:
studentmath.count()

studentid          395
gradeperiod1       395
gradeperiod2       395
gradeperiod3       395
school             395
sex                395
age                395
famsize            395
mothereducation    395
fathereducation    395
traveltime         395
studytime          395
freetime           395
failures           395
famrel             395
goout              395
dtype: int64

In [28]:
setvalues={
    "famrel":{1:"1:very bad",2:"2:bad",3:"3:neutral",4:"4:good",5:"5:excellent"},
    "freetime":{1:"1:very low",2:"2:low",3:"3:neutral",4:"4:high",5:"5:very high"},
    "goout":{1:"1:very low",2:"2:low",3:"3:neutral",4:"4:high",5:"5:very high"},
    "mothereducation":{0:np.nan,1:"1:k-4",2:"2:5-9",3:"3:secondary ed",4:"4:higher ed"},
    "fathereducation":{0:np.nan,1:"1:k-4",2:"2:5-9",3:"3:secondary ed",4:"4:higher ed"}
           }
studentmath.replace(setvalues, inplace=True)  
setvalueskeys = [k for k in setvalues]      

In [30]:
studentmath

,gradeperiod1,gradeperiod2,gradeperiod3,school,sex,age,famsize,mothereducation,fathereducation,traveltime,studytime,freetime,failures,famrel,goout
studentid,,,,,,,,,,,,,,,
001,5,6,6,GP,F,18,GT3,4:higher ed,4:higher ed,2,2,3:neutral,0,4:good,4:high
002,5,5,6,GP,F,17,GT3,1:k-4,1:k-4,1,2,3:neutral,0,5:excellent,3:neutral
003,7,8,10,GP,F,15,LE3,1:k-4,1:k-4,1,2,3:neutral,3,4:good,2:low
004,15,14,15,GP,F,15,GT3,4:higher ed,2:5-9,1,3,2:low,0,3:neutral,2:low
005,6,10,10,GP,F,16,GT3,3:secondary ed,3:secondary ed,1,2,3:neutral,0,4:good,2:low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,9,9,9,MS,M,20,LE3,2:5-9,2:5-9,1,2,5:very high,2,5:excellent,4:high
392,14,16,16,MS,M,17,LE3,3:secondary ed,1:k-4,2,1,4:high,0,2:bad,5:very high
393,10,8,7,MS,M,21,GT3,1:k-4,1:k-4,1,1,5:very high,3,5:excellent,3:neutral


In [31]:
studentmath[setvalueskeys].memory_usage(index=False)

famrel             3160
freetime           3160
goout              3160
mothereducation    3160
fathereducation    3160
dtype: int64

In [34]:
for col in studentmath[setvalueskeys].columns:
  studentmath[col] = studentmath[col].astype('category')

In [35]:
studentmath[setvalueskeys].memory_usage(index=False)

famrel             595
freetime           595
goout              595
mothereducation    587
fathereducation    587
dtype: int64

In [36]:
studentmath['famrel'].value_counts(sort=False, normalize=True)

1:very bad     0.020253
2:bad          0.045570
3:neutral      0.172152
4:good         0.493671
5:excellent    0.268354
Name: famrel, dtype: float64

In [38]:
studentmath[['freetime', 'goout']].apply(pd.Series.value_counts, sort=False, normalize=True)

,freetime,goout
1:very low,0.048101,0.058228
2:low,0.162025,0.260759
3:neutral,0.397468,0.329114
4:high,0.291139,0.217722
5:very high,0.101266,0.134177
